In [143]:
import pandas as pd
from datetime import datetime
import numpy as np

Información

- Obtener los medicamentos mas relevantes
- Intentar obtener lugar_residencia, lugar_procedencia, destino_alta y vive_solo --> No tenemos datos
- Ver como hago la columna gdiagalt --> De momento lo dejaria con los códigos, no hay una clasificación clara.


### Carga de archivos

In [144]:
df = pd.read_csv('../datos/DatosIdentificativosAdministrativos.csv')
df_constantes = pd.read_csv('../datos/ConstantesVitales.csv')
df_glucemias = pd.read_csv('../datos/GlucemiasGlucosurias.csv')
df_accidentes = pd.read_csv('../datos/AccidentesPlanta.csv')
df_dietas = pd.read_csv('../datos/DietasViasAlimentacion.csv')
df_antecedentes = pd.read_csv('../datos/ValoracionesAntecedentesPersonales.csv')
df_valoracion_enfermeria = pd.read_csv('../datos/ValoracionesEnfermeria.csv')
df_prescripcion_farmacos = pd.read_csv('../datos/PrescripcionFarmacos.csv')
df_ulceras = pd.read_csv('../datos/UlcerasPresion.csv')
#df_infecciones = pd.read_csv('../datos/InfeccionesNosocomiales.csv')
df_prestaciones =  pd.read_csv('../datos/Prestaciones.csv')
df_valoracion_medica = pd.read_csv('../datos/ValoracionesMedicas.csv')

/tmp/ipykernel_2315/247837475.py:2: DtypeWarning: Columns (16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_constantes = pd.read_csv('../datos/ConstantesVitales.csv')
/tmp/ipykernel_2315/247837475.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_glucemias = pd.read_csv('../datos/GlucemiasGlucosurias.csv')
/tmp/ipykernel_2315/247837475.py:8: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  df_prescripcion_farmacos = pd.read_csv('../datos/PrescripcionFarmacos.csv')
/tmp/ipykernel_2315/247837475.py:11: DtypeWarning: Columns (7,33,34,35,36,37,38,39,40,43,50,52,55,70,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df_prestaciones =  pd.read_csv('../datos/Prestaciones.csv')


### Datos identificaticos

In [145]:
# Eliminar las columnas que no tienen información relevante
df = df.drop(columns=['gprovinc', 'dnomprov', 'gpoblaci','dpoblaci','odiagini', 'odiagalt','dnomprov','ganoadme','gnumadme','gmotingr','dmotingr','gdiaging','gdiagini', 'gserulti', 'dnomserv','dsitalta','dmotalta','itraslad','odiaging','gdiasec1', 'odiasec1', 'gdiasec2', 'odiasec2','gmotalta'])
df.drop_duplicates(subset=['gidenpac'], keep='first', inplace=True)



In [146]:
# Reemplazar 'H' por 0 y 'M' por 1 en la columna 'itipsexo'
df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})


# Columna ds_izq_der para saber que lado de la cadera es 
# ------------------------------------------------------------------------------------------
# Eliminar las filas donde 'gdiagalt' no comienza con "S72."
df = df[df['gdiagalt'].str.startswith('S72.')]


# Eliminar las filas donde 'gdiagalt' contiene los códigos especificados
df = df.drop(df[df['gdiagalt'].isin(["S72.142D", "S72.90XD", "S72.102D"])].index)

# Definir el diccionario de mapeo
mapeo_fracturas = {
    'S72.141A': 'Intertrocantérea desplazada',
    'S72.012A': 'Intracapsular no especificada',
    'S72.142A': 'Intertrocantérea desplazada',
    'S72.011A': 'Intracapsular no especificada',
    'S72.002A': 'No especificada',
    'S72.001A': 'Trocantérea no especificado',
    'S72.102A': 'Trocantérea no especificado',
    'S72.101A': 'Trocantérea no especificado',
    'S72.22XA': 'Subtrocantérea desplazada',
    'S72.091A': 'Otras',
    'S72.112A': 'Trocantérea desplazada',
    'S72.8X1A': 'Otras',
    'S72.051A': 'No especificada',
    'S72.91XA': 'No especificada'
}

# Crear la nueva columna usando el mapeo
df['ds_tipo_fractura'] = df['gdiagalt'].map(mapeo_fracturas)

# Listas de códigos para "DERECHO" y "IZQUIERDO"
derecho_codes = ["S72.141A", "S72.011A", "S72.001A", "S72.101A", "S72.091A", "S72.8X1A", "S72.051A", "S72.91XA"]
izquierdo_codes = ["S72.012A", "S72.142A", "S72.002A", "S72.102A", "S72.22XA", "S72.112A", ]
# Categorizar la columna
def categorize_side(value):
    value = str(value)  # Convertir el valor a string por seguridad
    if any(code in value for code in izquierdo_codes):
        return 0
    elif any(code in value for code in derecho_codes):
        return 1
    else:
        return None  # Marcar para eliminar

# Lista de códigos a buscar
codigos_otras = ["S72.22XA", "S72.091A", "S72.112A", "S72.8X1A", "S72.051A", "S72.91XA"]

# Reemplaza los valores que coincidan en la columna 'gdiagalt'
df.loc[df['gdiagalt'].isin(codigos_otras), 'gdiagalt'] = "Otras"

# Crear la nueva columna 'ds_izq_der' basada en la función de categorización
df['ds_izq_der'] = df['gdiagalt'].apply(categorize_side)
df['ds_izq_der'] = pd.to_numeric(df['ds_izq_der'], errors='coerce').fillna(-999).astype(int)

# ------------------------------------------------------------------------------------------

# Columna turnos de trabajo
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fllegada' a formato de fecha y hora
df['fllegada'] = pd.to_datetime(df['fllegada'])

# Función para asignar el turno en función de la hora
def asignar_turno(hora):
    if 8 <= hora < 15:      #Mañana
        return 0
    elif 15 <= hora < 22:   # Tarde
        return 1
    else:                   # Noche
        return 2

# Crear una nueva columna 'ds_turno' basada en la hora de 'fllegada'
df['ds_turno'] = df['fllegada'].dt.hour.apply(asignar_turno)
# ------------------------------------------------------------------------------------------

# Columna edad
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fnacipac' a formato de fecha
df['fnacipac'] = pd.to_datetime(df['fnacipac'])

# Función para calcular la edad
def calcular_edad(fecha_nacimiento):
    hoy = datetime.now()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
    return edad

# Crear la nueva columna 'ds_edad' calculando la edad para cada fila
df['ds_edad'] = df['fnacipac'].apply(calcular_edad)
# ------------------------------------------------------------------------------------------

# Columnas de días estancia, preopertorio y postoperatorio
# ------------------------------------------------------------------------------------------
# Convertir las columnas de fecha a formato datetime, sin considerar las horas
df['fllegada'] = pd.to_datetime(df['fllegada']).dt.normalize()
df['faltplan'] = pd.to_datetime(df['faltplan']).dt.normalize()
df['finterve'] = pd.to_datetime(df['finterve']).dt.normalize()

# Calcular los días de estancia total (faltplan - fllegada)
df['ds_estancia'] = (df['faltplan'] - df['fllegada']).dt.days

# Calcular los días de preoperatorio incluyendo el día de intervención
df['ds_pre_oper'] = (df['finterve'] - df['fllegada']).dt.days
df['ds_pre_oper'] = pd.to_numeric(df['ds_pre_oper'], errors='coerce').fillna(-999).astype(int)


# Calcular los días de postoperatorio incluyendo el día de intervención
df['ds_post_oper'] = (df['faltplan'] - df['finterve']).dt.days
df['ds_post_oper'] = pd.to_numeric(df['ds_post_oper'], errors='coerce').fillna(-999).astype(int)

# ------------------------------------------------------------------------------------------

# Crear la columna 'ds_vivo_alta' basada en la condición de 'gsitalta'
df['ds_vivo_alta'] = df['gsitalta'].apply(lambda x: 0 if x == 7 else 1)

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ireingre'
df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})

# Crear la columna 'ds_dia_semana_llegada'(1 para lunes, 7 para domingo)
df['ds_dia_semana_llegada'] = df['fllegada'].dt.dayofweek + 1

# Crear la columna 'ds_mes_llegada'  
df['ds_mes_llegada'] = df['fllegada'].dt.month

# Obtenemos si viven en el centro o en las afueras
codigos_centro = [24001, 24002, 24003, 24004, 24005, 24006, 24007, 24008, 24009, 24010, 24012, 24070, 24071, 24080, ]  #Códigos del centro
df['ds_centro_afueras'] = df['gcodipos'].apply(lambda x: 1 if x in codigos_centro else 0)

# Reemplazar 'I' por 0 y 'U' por 1 en la columna 'itipingr'
df['itipingr'] = df['itipingr'].replace({'I': 0, 'U': 1})

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'iotrocen'
df['iotrocen'] = df['iotrocen'].replace({'N': 0, 'S': 1})

# Eliminar pacientes que no tienen nada que ver con Fractura de cadera
# Lista de valores que quieres eliminar
valores_a_eliminar = ['FDFD192912290', 'FRGR193404470', 'GNMR193911510']

# Eliminar filas donde gidenpac tenga valores en la lista
df = df[~df['gidenpac'].isin(valores_a_eliminar)]

# Eliminar columnas que ya no necesitamos
df = df.drop(columns=['faltplan'])
df = df.drop(columns=['fnacipac'])
#df = df.drop(columns=['fllegada'])
df = df.drop(columns=['finterve'])

/tmp/ipykernel_2315/2974921131.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})
/tmp/ipykernel_2315/2974921131.py:118: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})
/tmp/ipykernel_2315/2974921131.py:131: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behav

### Accidentes


In [147]:
# No tiene info que podamos usar
"""
# Filtrar df_accidentes para seleccionar solo las columnas relevantes
df_accidentes = df_accidentes[['gidenpac', 'ifamilia', 'gmovilid']]

# Realizar el merge con df, añadiendo las columnas de 'ifamilia' donde coincida 'gidenpac'
df = pd.merge(df, df_accidentes, on='gidenpac', how='left')

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ifamilia'
df['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})

# Cambiar el nombre de la columna 'ifamilia' a 'vive_solo'
df = df.rename(columns={'ifamilia': 'vive_solo'})

# Renombrar 'gmovilid' como 'movilidad_accidentes'
df = df.rename(columns={'gmovilid': 'movilidad_accidentes'})

# Eliminar filas duplicadas basándose en la columna 'gidenpac'
df = df.drop_duplicates(subset='gidenpac')
# Pasar a entero y añadimos -999 en los vacios
#df['vive_solo'] = df['vive_solo'].fillna(-999).astype(int)
#df['movilidad_accidentes'] = df['movilidad_accidentes'].fillna(-999).astype(int)
"""

"\n# Filtrar df_accidentes para seleccionar solo las columnas relevantes\ndf_accidentes = df_accidentes[['gidenpac', 'ifamilia', 'gmovilid']]\n\n# Realizar el merge con df, añadiendo las columnas de 'ifamilia' donde coincida 'gidenpac'\ndf = pd.merge(df, df_accidentes, on='gidenpac', how='left')\n\n# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ifamilia'\ndf['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})\n\n# Cambiar el nombre de la columna 'ifamilia' a 'vive_solo'\ndf = df.rename(columns={'ifamilia': 'vive_solo'})\n\n# Renombrar 'gmovilid' como 'movilidad_accidentes'\ndf = df.rename(columns={'gmovilid': 'movilidad_accidentes'})\n\n# Eliminar filas duplicadas basándose en la columna 'gidenpac'\ndf = df.drop_duplicates(subset='gidenpac')\n# Pasar a entero y añadimos -999 en los vacios\n#df['vive_solo'] = df['vive_solo'].fillna(-999).astype(int)\n#df['movilidad_accidentes'] = df['movilidad_accidentes'].fillna(-999).astype(int)\n"

### Constantes vitales

In [148]:
#La mayoria están vacios.
"""
# Seleccionar solo las columnas necesarias de df_constantes y eliminar duplicados tomando el primer valor no nulo
df_constantes = df_constantes[['gidenpac', 'ntensmin', 'ntensmax', 'ntempera', 'nsatuoxi']].dropna(how='all')
df_constantes = df_constantes.groupby('gidenpac').first().reset_index()

# Realizar el merge con df, añadiendo las constantes vitales
df = pd.merge(df, df_constantes, on='gidenpac', how='left')

df['ntensmin'] = pd.to_numeric(df['ntensmin'], errors='coerce').fillna(-999).astype(int)
df['ntensmax'] = pd.to_numeric(df['ntensmax'], errors='coerce').fillna(-999).astype(int)
df['ntempera'] = pd.to_numeric(df['ntempera'], errors='coerce').fillna(-999).astype(float)
df['nsatuoxi'] = pd.to_numeric(df['nsatuoxi'], errors='coerce').fillna(-999).astype(int)


# Pasamos a int y añadimos -999 en los vacios. En la temperatura dejamos float
#df['ntensmin'] = df['ntensmin'].fillna(-999).astype(int)
#df['ntensmax'] = df['ntensmax'].fillna(-999).astype(int)
#df['ntempera'] = df['ntempera'].fillna(-999).astype(float)
#df['nsatuoxi'] = df['nsatuoxi'].fillna(-999).astype(int)
"""

"\n# Seleccionar solo las columnas necesarias de df_constantes y eliminar duplicados tomando el primer valor no nulo\ndf_constantes = df_constantes[['gidenpac', 'ntensmin', 'ntensmax', 'ntempera', 'nsatuoxi']].dropna(how='all')\ndf_constantes = df_constantes.groupby('gidenpac').first().reset_index()\n\n# Realizar el merge con df, añadiendo las constantes vitales\ndf = pd.merge(df, df_constantes, on='gidenpac', how='left')\n\ndf['ntensmin'] = pd.to_numeric(df['ntensmin'], errors='coerce').fillna(-999).astype(int)\ndf['ntensmax'] = pd.to_numeric(df['ntensmax'], errors='coerce').fillna(-999).astype(int)\ndf['ntempera'] = pd.to_numeric(df['ntempera'], errors='coerce').fillna(-999).astype(float)\ndf['nsatuoxi'] = pd.to_numeric(df['nsatuoxi'], errors='coerce').fillna(-999).astype(int)\n\n\n# Pasamos a int y añadimos -999 en los vacios. En la temperatura dejamos float\n#df['ntensmin'] = df['ntensmin'].fillna(-999).astype(int)\n#df['ntensmax'] = df['ntensmax'].fillna(-999).astype(int)\n#df['nt

### Antecedentes


In [149]:
# Seleccionar solo las columnas necesarias de 'df_antecedentes' y crear una copia
df_ant = df_antecedentes[['gidenpac', 'dconclin', 'vbivalor']].copy()

# Crear columnas en 'df_ant' basadas en el contenido de 'dconclin'
df_ant['ds_HTA'] = df_ant["dconclin"].str.contains("HTA", na=False).astype(int)
df_ant['ds_alergia_medicamentosa'] = df_ant["dconclin"].str.contains("Alergia medicamentosa").astype(int)
df_ant['ds_alergia_alimenticia'] = df_ant["dconclin"].str.contains("Alergia alimenticia").astype(int)
df_ant['ds_diabetes'] = df_ant["dconclin"].str.contains("Diabetes Mellitus").astype(int)
df_ant['ds_otras_alergias'] = df_ant["dconclin"].str.contains("Otras alergias").astype(int)

# Actualizar valores en columnas de antecedentes usando 'vbivalor' si no es nulo
columns_to_update = ['ds_HTA', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias']
for col in columns_to_update:
    df_ant[col] = df_ant.apply(lambda x: x['vbivalor'] if pd.notnull(x['vbivalor']) and x[col] == 1 else x[col], axis=1)

# Reemplazar valores específicos ('N' -> 0, 'S' -> 1, 'A' -> 1) en todo el DataFrame
df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)

# Eliminar las columnas 'dconclin' y 'vbivalor' si ya no son necesarias
df_ant.drop(columns=['dconclin', 'vbivalor'], inplace=True)

# Eliminar duplicados en el DataFrame (si aún no se ha hecho)
df_ant = df_ant.drop_duplicates()

# Agrupar por 'gidenpac' y usar 'max' para que cualquier 1 en una columna mantenga el valor como 1
df_ant = df_ant.groupby('gidenpac', as_index=False).max()

# Realizar un merge entre df y df_ant sin sufijos
df = pd.merge(df, df_ant, on='gidenpac', how='left')

# Asignar valores de df_ant solo si son 1
for col in columns_to_update:
    df[col] = df.apply(lambda x: x[col] if pd.notnull(x[col]) and x[col] == 1 else np.nan if pd.isna(x[col]) else x[col], axis=1)


/tmp/ipykernel_2315/2776989086.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)


### Valoracion Enfermeria

In [150]:
import pandas as pd
import numpy as np

# Seleccionar solo las columnas necesarias de 'df_valoracion_enfermeria'
df_valoracion_enfermeria = df_valoracion_enfermeria[['gidenpac', 'dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor']].copy()

# Asegurarse de que 'ncodtabu', 'nvalncon', 'ovallcon', y 'vbivalor' sean numéricos
df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']] = df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']].apply(pd.to_numeric, errors='coerce')

# Definir los valores de 'dconclin' que se convertirán en nuevas columnas
patterns = {
    '_movilidad': "_Movilidad",
    'movilidad': "Movilidad",
    'lugar_residencia': "G Lugar de Residencia",
    'lugar_procedencia': "G Procedencia",
    'destino_alta': "G Destino Alta",
    'Barthel': "Resultado Indice de Barthel",
    'barthel_alta': "Resultado Indice de Barthel al Alta",
    'braden': "Resultado Escala de Braden",
    'riesgo_caida': "Resultado Escala Riesgo Caidas"
}

# Crear una expresión regular unida para filtrar las filas deseadas
regex_pattern = "|".join(patterns.values())

# Filtrar el DataFrame para conservar solo las filas que coinciden con el patrón
df_valoracion_enfermeria = df_valoracion_enfermeria[df_valoracion_enfermeria['dconclin'].str.contains(regex_pattern, na=False)]

# Iterar sobre el diccionario para crear las columnas basadas en el valor máximo entre las columnas especificadas
for new_col, pattern in patterns.items():
    # Crear la nueva columna con el valor máximo de las columnas seleccionadas donde 'dconclin' coincide con el patrón
    df_valoracion_enfermeria[new_col] = np.where(
        df_valoracion_enfermeria['dconclin'].str.contains(pattern, na=False),
        df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']].max(axis=1),
        np.nan
    )

# Agrupar por 'gidenpac' y mantener el valor máximo en cada columna para eliminar duplicados
df_valoracion_enfermeria = df_valoracion_enfermeria.groupby('gidenpac', as_index=False).max()

# Eliminar las columnas no deseadas del DataFrame
df_valoracion_enfermeria = df_valoracion_enfermeria.drop(columns=['dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor', '_movilidad','barthel_alta'])

# Combinar df y df_valoracion_enfermeria por la columna 'gidenpac'
df= pd.merge(df, df_valoracion_enfermeria, on='gidenpac', how='left')

# Convertimos a enteros y los Nan los pasamos a -999
df['movilidad'] = df['movilidad'].fillna(-999).astype(int)
df['lugar_residencia'] = df['lugar_residencia'].fillna(-999).astype(int)
df['lugar_procedencia'] = df['lugar_procedencia'].fillna(-999).astype(int)
df['destino_alta'] = df['destino_alta'].fillna(-999).astype(int)
df['Barthel'] = df['Barthel'].fillna(-999).astype(int)
df['braden'] = df['braden'].fillna(-999).astype(int)
df['riesgo_caida'] = df['riesgo_caida'].fillna(-999).astype(int)


### Glucemias

In [151]:
# Pocos datos

# Crear un conjunto con los valores de gidenpac en df_glucemias
glucemias_pacientes = set(df_glucemias['gidenpac'].dropna())

# Crear la columna 'ds_diabetes' solo en las casillas vacías. Añadimos info a la casilla ds_diabetes
df['ds_diabetes'] = df.apply(lambda row: 1 if pd.isna(row['ds_diabetes']) and row['gidenpac'] in glucemias_pacientes 
                             else (0 if pd.isna(row['ds_diabetes']) else row['ds_diabetes']), axis=1)


### Valoraciones Medicas

In [152]:
# Definir los códigos de 'gcodconc' que queremos filtrar para los diagnosticos
codigos = [1368, 1369, 1370, 1371, 1372, 1373, 1374, 1742, 1743, 1750, 1813, 1814, 1815]

# Listado de archivos a procesar
archivos = [
    '../datos/ValoracionesMedicas.csv' 
]

# Lista para almacenar los DataFrames filtrados
dataframes_filtrados = []

# Iterar sobre cada archivo
for archivo in archivos:
    # Cargar el archivo
    df_v_med = pd.read_csv(archivo)
    
    # Filtrar las filas donde 'gcodconc' esté en la lista de códigos
    df_filtrado = df_v_med[df_v_med['gcodconc'].isin(codigos)]
    
    # Seleccionar solo las columnas 'gidenpac', 'gcodconc' y 'ovallcon'
    df_filtrado = df_filtrado[['gidenpac', 'gcodconc', 'ovallcon']]
    
    # Añadir el DataFrame filtrado a la lista
    dataframes_filtrados.append(df_filtrado)

# Concatenar todos los DataFrames filtrados en uno solo
df_val_medicas_combined = pd.concat(dataframes_filtrados, ignore_index=True)


# Guardar el resultado en un archivo CSV
df_val_medicas_combined.to_csv('DF_VAL_MEDICAS_COMBINED.csv', index=False)

conteo_codigos = df_val_medicas_combined['ovallcon'].value_counts()



In [153]:
# Crear la nueva columna 'ds_HTA' para detectar 'HTA' en cualquier parte del texto
df_val_medicas_combined['ds_HTA'] = df_val_medicas_combined['ovallcon'].str.contains('HTA', case=False, na=False).astype(int)
df_val_medicas_combined['ds_ITU'] = df_val_medicas_combined['ovallcon'].str.contains('ITU', case=False, na=False).astype(int)
#df_val_medicas_combined['ds_acido_folico'] = df_val_medicas_combined['ovallcon'].str.contains('ACIDO FOLICO', case=False, na=False).astype(int)
df_val_medicas_combined['ds_anemia'] = df_val_medicas_combined['ovallcon'].str.contains('ANEMIA', case=False, na=False).astype(int)
df_val_medicas_combined['ds_vitamina_d'] = df_val_medicas_combined['ovallcon'].str.contains('VITAMINA D', case=False, na=False).astype(int)
#df_val_medicas_combined['ds_obesidad'] = df_val_medicas_combined['ovallcon'].str.contains('OBESIDAD', case=False, na=False).astype(int)
#df_val_medicas_combined['ds_osteoporosis'] = df_val_medicas_combined['ovallcon'].str.contains('OSTEOPOROSIS', case=False, na=False).astype(int)


# Insuficiencia respiratoria
# ___________________________________________________________
respiratory_terms = [
    'INSUFICIENCIA RESPIRATORIA AGUDA', 'INSUFICIENCIA RESPIRATORIA', 
    'INSUFICIENCIA RESPIRATORIA AGUDA PARCIAL', 'INSUFICIENCIA RESPIRATORIA CRÓNICA AGUDIZADA', 
    'INSUFICIENCIA RESPIRATORIA PARCIAL', 'INSUFICIENCIA RESPIRATORIA 2RIA', 
    'INSUFICIENCIA RESPIRATORIA GLOBAL', 'INSUFICIENCIA RESPIRATORIA AGUDA SECUNDARIA', 
    'INSUFICIENCIA RESPIRATORIA PARCIAL CRÓNICA AGUDIZADA', 
    'INSUFICIENCIA RESPIRATORIA AGUDA PARCIAL RESUELTA', 'EPOC AGUDIZADO', 'EPOC', 'EPOC REAGUDIZADO'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(respiratory_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_insuficiencia_respiratoria'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Diabetes
# ___________________________________________________________
diabetes_terms = [
    'DM TIPO 2 ', 'DM TIPO II', 'DIABETES MELLITUS', 'DM-II',  'DIABETES MELLITUS TIPO II','DIABETES'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(diabetes_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_diabetes'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Infección respiratoria
# ___________________________________________________________
infeccion_respiratoria_terms = [
   'INFECCIÓN RESPIRATORIA', 'INFECCIÓN RESPIRATORIA DE VÍAS BAJAS',  'INFECCIÓN RESPIRATORIA DE VÍAS INFERIORES', 'INFECCION RESPIRATORIA', 'INFECCION RESPIRATORIA AGUDA', 'INFECCIÓN RESPIRATORIA AGUDA', 'INFECCIÓN RESPIRATORIA DE VÍAS BAJAS NO CONDENSANTE'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(infeccion_respiratoria_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
#df_val_medicas_combined['ds_infeccion_respiratoria'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Insuficiencia Cardiaca
# ___________________________________________________________
insuficiencia_cardiaca_terms = [
'INSUFICIENCIA CARDIACA', 'INSUFICIENCIA CARDIACA DESCOMPENSADA',  'INSUFICIENCIA RESPIRATORIA CRÓNICA', 'INSUFICIENCIA CARDIACA CONGESTIVA',  'I. CARDIACA', 'INSUFICIENCIA CARDIACA CONGESTIVA DESCOMPENSADA', 'INSUFICIENCIA CARDÍACA', 'INSUFICIENCIA CARDIACA CRÓNICA DESCOMPENSADA',  'INSUFICIENCIA CARDIACA CONGESTIVA AGUDA', 'ICC AGUDA', 'CARDIOPATIA ISQUEMICA',  'CARDIOPATÍA ISQUÉMICA', 'ICC', 'ICC DESCOMPENSADA']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(insuficiencia_cardiaca_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_insuficiencia_cardiaca'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Deterioro cognitivo
# ___________________________________________________________
deterioro_cognitivo_terms = [
	'DETERIORO COGNITIVO', 'DETERIORO COGNITIVO', 'DEMENCIA']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(deterioro_cognitivo_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_deterioro_cognitivo'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Exitus
# ___________________________________________________________
exitus_terms = [
    'EXITUS', 'ÉXITUS']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(exitus_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_exitus'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Insuficiencia renal
# ___________________________________________________________
insuficiencia_renal_terms = [
    'I. RENAL', 'INSUFICIENCIA RENAL', 'ENFERMEDAD RENAL CRÓNICA', 'INSUFICIENCIA RENAL AGUDA', 'INSUFICIENCIA RENAL CRÓNICA AGUDIZADA', 'INSUFICIENCIA RENAL CRONICA', 'IRC', 'ERC REAGUDIZADA', 'ERC AGUDIZADA', 'ERC']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(insuficiencia_renal_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_insuficiencia_renal'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________


# Eliminamos las columnas que ya no necesitamos
df_filtrado = df_val_medicas_combined.drop(columns=['gcodconc', 'ovallcon'])

# Agrupar por 'gidenpac' y aplicar max() para cada columna
df_val_medicas_procesado = df_filtrado.groupby('gidenpac').max().reset_index()


df_val_medicas_procesado.to_csv('DF_VAL_MEDICAS_PROCESED.csv', index=False)



### Filtrado de los datos finales

Unir el DataFrame principal (df) con el de valoraciones medicas ya filtrado y procesado (df_val_medicas_procesado)

In [154]:
# Unir df y df_val_medicas_procesado por la columna 'gidenpac'
df = df.merge(df_val_medicas_procesado, on='gidenpac', how='left')
# Reemplazar valores NaN o nulos por -999
#df.fillna(-999, inplace=True)

# Convertimos a enteros y los Nan los pasamos a -999
"""
df['ds_vitamina_d'] = df['ds_vitamina_d'].fillna(-999).astype(int)
df['ds_obesidad'] = df['ds_obesidad'].fillna(-999).astype(int)
df['ds_osteoporosis'] = df['ds_osteoporosis'].fillna(-999).astype(int)
df['ds_insuficiencia_respiratoria'] = df['ds_insuficiencia_respiratoria'].fillna(-999).astype(int)
df['ds_diabetes_y'] = df['ds_diabetes_y'].fillna(-999).astype(int)
df['ds_infeccion_respiratoria'] = df['ds_infeccion_respiratoria'].fillna(-999).astype(int)
df['ds_insuficiencia_cardiaca'] = df['ds_insuficiencia_cardiaca'].fillna(-999).astype(int)
df['ds_deterioro_cognitivo'] = df['ds_deterioro_cognitivo'].fillna(-999).astype(int)
df['ds_insuficiencia_renal'] = df['ds_insuficiencia_renal'].fillna(-999).astype(int)
df['ds_HTA_y'] = df['ds_HTA_y'].fillna(-999).astype(int)
df['ds_ITU'] = df['ds_ITU'].fillna(-999).astype(int)
df['ds_acido_folico'] = df['ds_acido_folico'].fillna(-999).astype(int)
df['ds_anemia'] = df['ds_anemia'].fillna(-999).astype(int)
"""

"\ndf['ds_vitamina_d'] = df['ds_vitamina_d'].fillna(-999).astype(int)\ndf['ds_obesidad'] = df['ds_obesidad'].fillna(-999).astype(int)\ndf['ds_osteoporosis'] = df['ds_osteoporosis'].fillna(-999).astype(int)\ndf['ds_insuficiencia_respiratoria'] = df['ds_insuficiencia_respiratoria'].fillna(-999).astype(int)\ndf['ds_diabetes_y'] = df['ds_diabetes_y'].fillna(-999).astype(int)\ndf['ds_infeccion_respiratoria'] = df['ds_infeccion_respiratoria'].fillna(-999).astype(int)\ndf['ds_insuficiencia_cardiaca'] = df['ds_insuficiencia_cardiaca'].fillna(-999).astype(int)\ndf['ds_deterioro_cognitivo'] = df['ds_deterioro_cognitivo'].fillna(-999).astype(int)\ndf['ds_insuficiencia_renal'] = df['ds_insuficiencia_renal'].fillna(-999).astype(int)\ndf['ds_HTA_y'] = df['ds_HTA_y'].fillna(-999).astype(int)\ndf['ds_ITU'] = df['ds_ITU'].fillna(-999).astype(int)\ndf['ds_acido_folico'] = df['ds_acido_folico'].fillna(-999).astype(int)\ndf['ds_anemia'] = df['ds_anemia'].fillna(-999).astype(int)\n"

Vamos a combinar las 2 columnas de HTA.
- HTA: Tenemos 2. X e Y. 
	- X viene de antecedentes
	- Y de valoraciones médicas
- Podemos combinar las 2, ya que es más probable que no pongan la info, a poner una info falsa


In [155]:
df['ds_HTA'] = df[['ds_HTA_x', 'ds_HTA_y']].max(axis=1)

# Eliminar las columnas originales
df.drop(columns=['ds_HTA_x', 'ds_HTA_y'], inplace=True)


Hacemos lo mismo con la diabetes


In [156]:
df['ds_diabetes'] = df[['ds_diabetes_x', 'ds_diabetes_y']].max(axis=1)

# Eliminar las columnas originales
df.drop(columns=['ds_diabetes_x', 'ds_diabetes_y'], inplace=True)

Eliminar columnas que no nos van a servir


In [157]:
df.drop(columns=['ds_exitus'], inplace=True)    # Tenemos la varibale vivo al alta (ds_vivo_alta)
df.drop(columns=['gcodipos'], inplace=True)  # La elimino porque tengo el codigo postal (gcodpos)

Eliminamos filas que les falten datos importantes

In [158]:

df = df[df['ds_pre_oper'] != -999]  # Eliminamos filas que no tengan los dias de pre o post operacion
df = df[df['ds_post_oper'] != -999]
df = df[df['ds_izq_der'] != -999]


"""
# Si las valoraciones médicas no tienen info sobre el paciente, lo eliminamos
columnas_a_verificar = [
    'ds_ITU', 'ds_acido_folico', 'ds_anemia', 'ds_vitamina_d', 
    'ds_obesidad', 'ds_osteoporosis', 'ds_insuficiencia_respiratoria', 
    'ds_infeccion_respiratoria', 'ds_insuficiencia_cardiaca', 
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal'
]

df = df[~df[columnas_a_verificar].isin([-999]).any(axis=1)]
"""

"\n# Si las valoraciones médicas no tienen info sobre el paciente, lo eliminamos\ncolumnas_a_verificar = [\n    'ds_ITU', 'ds_acido_folico', 'ds_anemia', 'ds_vitamina_d', \n    'ds_obesidad', 'ds_osteoporosis', 'ds_insuficiencia_respiratoria', \n    'ds_infeccion_respiratoria', 'ds_insuficiencia_cardiaca', \n    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal'\n]\n\ndf = df[~df[columnas_a_verificar].isin([-999]).any(axis=1)]\n"

### Prescripción Farmacos

In [159]:
"""# Definir las columnas que deseas conservar
columnas_deseadas = ['gidenpac', 'fapuntes', 'finitrat', 'gfarmaco', 'tarasist']

# Filtrar solo las filas donde el valor de gidenpac esté presente en df['gidenpac']
df_prescripcion_farmacos = df_prescripcion_farmacos[df_prescripcion_farmacos['gidenpac'].isin(df['gidenpac'])][columnas_deseadas]



df_prescripcion_farmacos.head()
df_prescripcion_farmacos.info()
"""

"# Definir las columnas que deseas conservar\ncolumnas_deseadas = ['gidenpac', 'fapuntes', 'finitrat', 'gfarmaco', 'tarasist']\n\n# Filtrar solo las filas donde el valor de gidenpac esté presente en df['gidenpac']\ndf_prescripcion_farmacos = df_prescripcion_farmacos[df_prescripcion_farmacos['gidenpac'].isin(df['gidenpac'])][columnas_deseadas]\n\n\n\ndf_prescripcion_farmacos.head()\ndf_prescripcion_farmacos.info()\n"

In [160]:
#df_prescripcion_farmacos.head()


In [161]:
#df.head()

In [162]:
"""df['fllegada'] = pd.to_datetime(df['fllegada']).dt.date
df_prescripcion_farmacos['fapuntes'] = pd.to_datetime(df_prescripcion_farmacos['fapuntes']).dt.date
df_prescripcion_farmacos['finitrat'] = pd.to_datetime(df_prescripcion_farmacos['finitrat']).dt.date

df_prescripcion_farmacos.head()
"""

"df['fllegada'] = pd.to_datetime(df['fllegada']).dt.date\ndf_prescripcion_farmacos['fapuntes'] = pd.to_datetime(df_prescripcion_farmacos['fapuntes']).dt.date\ndf_prescripcion_farmacos['finitrat'] = pd.to_datetime(df_prescripcion_farmacos['finitrat']).dt.date\n\ndf_prescripcion_farmacos.head()\n"

In [163]:
'''import pandas as pd

# Convertir las columnas de fechas a datetime y extraer solo la parte de la fecha
df['fllegada'] = pd.to_datetime(df['fllegada']).dt.date
df_prescripcion_farmacos['fapuntes'] = pd.to_datetime(df_prescripcion_farmacos['fapuntes']).dt.date
df_prescripcion_farmacos['finitrat'] = pd.to_datetime(df_prescripcion_farmacos['finitrat']).dt.date

# Crear un diccionario que asocie a cada paciente (gidenpac) una lista de sus fechas de llegada
fechas_por_paciente = df.groupby('gidenpac')['fllegada'].apply(list).to_dict()

def toma_medicamento(row):
    """
    Para cada registro en df_prescripcion_farmacos, se verifica si el paciente tiene
    alguna fecha de llegada (fllegada en df) que se encuentre dentro del intervalo [fapuntes, finitrat].
    Si es así, se conserva la fila; en caso contrario, se elimina.
    """
    gidenpac = row['gidenpac']
    # Si el paciente no tiene registro en df, se elimina la fila
    if gidenpac not in fechas_por_paciente:
        return False
    # Comprobar si existe alguna fecha en el intervalo [fapuntes, finitrat]
    return any(row['fapuntes'] <= fecha <= row['finitrat'] for fecha in fechas_por_paciente[gidenpac])

# Filtrar las filas que cumplen la condición
df_prescripcion_farmacos_filtrado = df_prescripcion_farmacos[df_prescripcion_farmacos.apply(toma_medicamento, axis=1)]

# Opcional: conservar solo las columnas deseadas
columnas_deseadas = ['gidenpac', 'fapuntes', 'finitrat', 'gfarmaco', 'tarasist']
df_prescripcion_farmacos_filtrado = df_prescripcion_farmacos_filtrado[columnas_deseadas]

#print(df_prescripcion_farmacos_filtrado)
df_prescripcion_farmacos_filtrado.info()

'''

'import pandas as pd\n\n# Convertir las columnas de fechas a datetime y extraer solo la parte de la fecha\ndf[\'fllegada\'] = pd.to_datetime(df[\'fllegada\']).dt.date\ndf_prescripcion_farmacos[\'fapuntes\'] = pd.to_datetime(df_prescripcion_farmacos[\'fapuntes\']).dt.date\ndf_prescripcion_farmacos[\'finitrat\'] = pd.to_datetime(df_prescripcion_farmacos[\'finitrat\']).dt.date\n\n# Crear un diccionario que asocie a cada paciente (gidenpac) una lista de sus fechas de llegada\nfechas_por_paciente = df.groupby(\'gidenpac\')[\'fllegada\'].apply(list).to_dict()\n\ndef toma_medicamento(row):\n    """\n    Para cada registro en df_prescripcion_farmacos, se verifica si el paciente tiene\n    alguna fecha de llegada (fllegada en df) que se encuentre dentro del intervalo [fapuntes, finitrat].\n    Si es así, se conserva la fila; en caso contrario, se elimina.\n    """\n    gidenpac = row[\'gidenpac\']\n    # Si el paciente no tiene registro en df, se elimina la fila\n    if gidenpac not in fechas_p

In [164]:
'''
# Conteo de cada valor en la columna gfarmaco
conteo_gfarmaco = df_prescripcion_farmacos_filtrado['gfarmaco'].value_counts()
conteo_gfarmaco_filtrado = conteo_gfarmaco[conteo_gfarmaco >= 10]
print("Conteo de gfarmaco (solo valores >= 10):")
print(conteo_gfarmaco_filtrado)

# Conteo de cada valor en la columna tarasist
conteo_tarasist = df_prescripcion_farmacos_filtrado['tarasist'].value_counts()
conteo_tarasist_filtrado = conteo_tarasist[conteo_tarasist >= 10]
print("\nConteo de tarasist (solo valores >= 10):")
print(conteo_tarasist_filtrado)

'''

'\n# Conteo de cada valor en la columna gfarmaco\nconteo_gfarmaco = df_prescripcion_farmacos_filtrado[\'gfarmaco\'].value_counts()\nconteo_gfarmaco_filtrado = conteo_gfarmaco[conteo_gfarmaco >= 10]\nprint("Conteo de gfarmaco (solo valores >= 10):")\nprint(conteo_gfarmaco_filtrado)\n\n# Conteo de cada valor en la columna tarasist\nconteo_tarasist = df_prescripcion_farmacos_filtrado[\'tarasist\'].value_counts()\nconteo_tarasist_filtrado = conteo_tarasist[conteo_tarasist >= 10]\nprint("\nConteo de tarasist (solo valores >= 10):")\nprint(conteo_tarasist_filtrado)\n\n'

In [165]:
'''
# Obtener los valores que aparecen 10 veces o más para gfarmaco y tarasist
valores_validos_gfarmaco = conteo_gfarmaco_filtrado.index
valores_validos_tarasist = conteo_tarasist_filtrado.index

# Filtrar el DataFrame para que solo se conserven las filas con esos valores
df_prescripcion_farmacos_filtrado = df_prescripcion_farmacos_filtrado[
    df_prescripcion_farmacos_filtrado['gfarmaco'].isin(valores_validos_gfarmaco) &
    df_prescripcion_farmacos_filtrado['tarasist'].isin(valores_validos_tarasist)
]


df_prescripcion_farmacos_filtrado.info()
df_prescripcion_farmacos.head()
'''

"\n# Obtener los valores que aparecen 10 veces o más para gfarmaco y tarasist\nvalores_validos_gfarmaco = conteo_gfarmaco_filtrado.index\nvalores_validos_tarasist = conteo_tarasist_filtrado.index\n\n# Filtrar el DataFrame para que solo se conserven las filas con esos valores\ndf_prescripcion_farmacos_filtrado = df_prescripcion_farmacos_filtrado[\n    df_prescripcion_farmacos_filtrado['gfarmaco'].isin(valores_validos_gfarmaco) &\n    df_prescripcion_farmacos_filtrado['tarasist'].isin(valores_validos_tarasist)\n]\n\n\ndf_prescripcion_farmacos_filtrado.info()\ndf_prescripcion_farmacos.head()\n"

In [166]:
'''
# Asegurarse de que la columna gfarmaco es de tipo string
df_prescripcion_farmacos_filtrado['gfarmaco'] = df_prescripcion_farmacos_filtrado['gfarmaco'].astype(str)

# Crear variables dummy para la columna gfarmaco
dummies_gfarmaco = pd.get_dummies(df_prescripcion_farmacos_filtrado['gfarmaco'], prefix='gfarmaco')

# Concatenar las variables dummy al DataFrame original
df_prescripcion_farmacos_filtrado = pd.concat([df_prescripcion_farmacos_filtrado, dummies_gfarmaco], axis=1)

# Visualizar el resultado
print(df_prescripcion_farmacos_filtrado.head())
df_prescripcion_farmacos_filtrado.info()
'''

"\n# Asegurarse de que la columna gfarmaco es de tipo string\ndf_prescripcion_farmacos_filtrado['gfarmaco'] = df_prescripcion_farmacos_filtrado['gfarmaco'].astype(str)\n\n# Crear variables dummy para la columna gfarmaco\ndummies_gfarmaco = pd.get_dummies(df_prescripcion_farmacos_filtrado['gfarmaco'], prefix='gfarmaco')\n\n# Concatenar las variables dummy al DataFrame original\ndf_prescripcion_farmacos_filtrado = pd.concat([df_prescripcion_farmacos_filtrado, dummies_gfarmaco], axis=1)\n\n# Visualizar el resultado\nprint(df_prescripcion_farmacos_filtrado.head())\ndf_prescripcion_farmacos_filtrado.info()\n"

In [167]:
'''import pandas as pd

# Eliminar las columnas 'fapuntes' y 'finitrat'
df_prescripcion_farmacos_filtrado = df_prescripcion_farmacos_filtrado.drop(columns=['fapuntes', 'finitrat'])

# Asegurarse de que la columna gfarmaco es de tipo string
df_prescripcion_farmacos_filtrado['gfarmaco'] = df_prescripcion_farmacos_filtrado['gfarmaco'].astype(str)

# Crear variables dummy para la columna gfarmaco
dummies_gfarmaco = pd.get_dummies(df_prescripcion_farmacos_filtrado['gfarmaco'], prefix='gfarmaco')

# Concatenar solo la columna gidenpac y las dummies
df_with_dummies = pd.concat([df_prescripcion_farmacos_filtrado[['gidenpac']], dummies_gfarmaco], axis=1)

# Agrupar por gidenpac usando el máximo (si aparece el medicamento en alguna fila, se marca como 1)
df_resultado_medicamentos = df_with_dummies.groupby('gidenpac', as_index=False).max()

# Convertir las columnas dummy a enteros para que sean 0 o 1
dummy_cols = [col for col in df_resultado_medicamentos.columns if col.startswith('gfarmaco_')]
df_resultado_medicamentos[dummy_cols] = df_resultado_medicamentos[dummy_cols].astype(int)

# Visualizar el resultado
print(df_resultado_medicamentos.head())

# Exportar a CSV
df_resultado_medicamentos.to_csv('df_resultado_medicamentos.csv', index=False)
'''


"import pandas as pd\n\n# Eliminar las columnas 'fapuntes' y 'finitrat'\ndf_prescripcion_farmacos_filtrado = df_prescripcion_farmacos_filtrado.drop(columns=['fapuntes', 'finitrat'])\n\n# Asegurarse de que la columna gfarmaco es de tipo string\ndf_prescripcion_farmacos_filtrado['gfarmaco'] = df_prescripcion_farmacos_filtrado['gfarmaco'].astype(str)\n\n# Crear variables dummy para la columna gfarmaco\ndummies_gfarmaco = pd.get_dummies(df_prescripcion_farmacos_filtrado['gfarmaco'], prefix='gfarmaco')\n\n# Concatenar solo la columna gidenpac y las dummies\ndf_with_dummies = pd.concat([df_prescripcion_farmacos_filtrado[['gidenpac']], dummies_gfarmaco], axis=1)\n\n# Agrupar por gidenpac usando el máximo (si aparece el medicamento en alguna fila, se marca como 1)\ndf_resultado_medicamentos = df_with_dummies.groupby('gidenpac', as_index=False).max()\n\n# Convertir las columnas dummy a enteros para que sean 0 o 1\ndummy_cols = [col for col in df_resultado_medicamentos.columns if col.startswith(

In [168]:
'''df_resultado_medicamentos.info()
print(df_resultado_medicamentos)
df_resultado_medicamentos.to_csv('df_medicamentos_dummy.csv', index=False)
'''


"df_resultado_medicamentos.info()\nprint(df_resultado_medicamentos)\ndf_resultado_medicamentos.to_csv('df_medicamentos_dummy.csv', index=False)\n"

In [169]:
'''
# Unir df con df_resultado usando la columna gidenpac
df = pd.merge(df, df_resultado_medicamentos, on='gidenpac', how='left')

# En caso de que algún paciente no tenga registros en df_resultado, rellenamos con 0
dummy_cols = [col for col in df_resultado_medicamentos.columns if col.startswith('gfarmaco_')]
df[dummy_cols] = df[dummy_cols].fillna(0).astype(int)

print(df.head())
df.info()
'''

"\n# Unir df con df_resultado usando la columna gidenpac\ndf = pd.merge(df, df_resultado_medicamentos, on='gidenpac', how='left')\n\n# En caso de que algún paciente no tenga registros en df_resultado, rellenamos con 0\ndummy_cols = [col for col in df_resultado_medicamentos.columns if col.startswith('gfarmaco_')]\ndf[dummy_cols] = df[dummy_cols].fillna(0).astype(int)\n\nprint(df.head())\ndf.info()\n"

In [170]:
# Medicamentos prescitos
# Se podria usar? 
"""
# Seleccionar solo las columnas 'gidenpac' y 'tarasist'
df_prescripcion_farmacos = df_prescripcion_farmacos[['gidenpac', 'tarasist']]
df_prescripcion_farmacos = df_prescripcion_farmacos[['gidenpac', 'tarasist']]

# Combinar los dos DataFrames
df_combined = pd.concat([df_prescripcion_farmacos, df_prescripcion_farmacos], ignore_index=True)

# Agrupar por 'gidenpac' y combinar los medicamentos en una lista
df_medicamentos = df_combined.groupby('gidenpac')['tarasist'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()

# Verificar el resultado
print(df_medicamentos.head())

df_medicamentos.to_csv('medicamentos.csv')
"""

"\n# Seleccionar solo las columnas 'gidenpac' y 'tarasist'\ndf_prescripcion_farmacos = df_prescripcion_farmacos[['gidenpac', 'tarasist']]\ndf_prescripcion_farmacos = df_prescripcion_farmacos[['gidenpac', 'tarasist']]\n\n# Combinar los dos DataFrames\ndf_combined = pd.concat([df_prescripcion_farmacos, df_prescripcion_farmacos], ignore_index=True)\n\n# Agrupar por 'gidenpac' y combinar los medicamentos en una lista\ndf_medicamentos = df_combined.groupby('gidenpac')['tarasist'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()\n\n# Verificar el resultado\nprint(df_medicamentos.head())\n\ndf_medicamentos.to_csv('medicamentos.csv')\n"

Finalizando el preprocesado

In [171]:
# Convertir a categorico
columnas_a_convertir = ['itipsexo','ireingre','itipingr','iotrocen', 'ds_centro_afueras', 'movilidad', 'riesgo_caida','gsitalta','gdiagalt','ds_izq_der','ds_turno','ds_vivo_alta','ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_HTA','ds_alergia_medicamentosa','ds_alergia_alimenticia','ds_diabetes','ds_otras_alergias', 'lugar_residencia','lugar_procedencia','destino_alta','ds_ITU','ds_anemia','ds_vitamina_d','ds_insuficiencia_respiratoria','ds_insuficiencia_cardiaca','ds_deterioro_cognitivo','ds_insuficiencia_renal' ]
df[columnas_a_convertir] = df[columnas_a_convertir].astype('category')

In [172]:
# Eliminamos columnas que pensabamos usar pero apenas contienen datos:
df = df.drop(columns=['ds_tipo_fractura','destino_alta', 'lugar_procedencia', 'lugar_residencia', 'itipingr', 'ireingre'])

#Eliminamos pacientes con "gsitalta" igual a 3,4 o 5, porque apenas tienen datos
# Filtrar el dataframe para conservar solo las filas que NO tengan gsitalta igual a 3, 4 o 5
df = df[~df["gsitalta"].isin([3, 4, 5])]
df["gsitalta"] = df["gsitalta"].cat.remove_unused_categories()


#Eliminamos pacientes con "movilidad" igual a 1 o 4, porque apenas tienen datos
df = df[~df["movilidad"].isin([1, 4])]
df["movilidad"] = df["movilidad"].cat.remove_unused_categories()



In [173]:
def transformar(valor):
    # Convertir el valor a string y eliminar espacios, además de pasarlo a minúsculas
    s = str(valor).strip().lower()
    if s in ["", "nan"]:
        return 0
    try:
        v = float(s)
        if v == 0:
            return 0
        elif v == 1:
            return 1
        else:
            return 0
    except Exception:
        return 0

# Lista de columnas a modificar
cols = [
    "ds_ITU", "ds_anemia", "ds_vitamina_d", "ds_insuficiencia_respiratoria",
    "ds_insuficiencia_cardiaca", "ds_deterioro_cognitivo", "ds_insuficiencia_renal",
    "ds_HTA", "ds_diabetes"
]

# Convertir la columna a string antes de aplicar la función de transformación
for col in cols:
    df[col] = df[col].astype(str).apply(transformar).astype('category')



In [174]:

comorbidity_cols = [
    "ds_ITU", "ds_anemia", "ds_vitamina_d", "ds_insuficiencia_respiratoria",
    "ds_insuficiencia_cardiaca", "ds_deterioro_cognitivo", "ds_insuficiencia_renal",
    "ds_HTA", "ds_diabetes"
]

# 1. Convertir a numérico las columnas de comorbilidades para poder sumar
for col in comorbidity_cols:
    if col not in df.columns:
        print(f"Advertencia: La columna {col} no se encontró en el dataset.")
    else:
        df[col] = pd.to_numeric(df[col].astype(str), errors='coerce')

# 2. Crear la nueva característica sumando las comorbilidades
df["ds_comorbilidades"] = df[comorbidity_cols].sum(axis=1)

# 3. Volver a convertir las columnas de comorbilidades a tipo 'category'
for col in comorbidity_cols:
    if col in df.columns:
        df[col] = df[col].astype("category")



AÑADIMOS ULCERAS POR PRESION E INFECCIONES NOSOCOMIALES


In [175]:
# Añadir la nueva columna
df['ds_ulcera_presion'] = df['gidenpac'].isin(df_ulceras['gidenpac']).astype(int)
#df['df_infecciones_nosocomiales'] = df['gidenpac'].isin(df_infecciones['gidenpac']).astype(int)


In [176]:
# Añadimos el médico
# Primero, filtramos df_prestaciones para obtener solo las filas donde gsersoli es 23342
df_prestaciones_filtrado = df_prestaciones[df_prestaciones['gsersoli'] == 23342]

# Luego, hacemos un merge de df con df_prestaciones_filtrado
df = df.merge(df_prestaciones_filtrado[['gidenpac', 'gpersoli']], 
              on='gidenpac', 
              how='left')

# Rellenamos los valores NaN en la columna medico (si los hay)
df['gpersoli'] = df['gpersoli'].fillna('')
df['gpersoli'] = pd.Categorical(df['gpersoli'])



In [177]:
df.to_csv('DATOS_PREPROCESADOS_COMPLETO.csv', index=False)

In [178]:
for col in df.columns:
    print(f"Columna: {col}")
    print("Cantidad de cada valor:")
    print(df[col].value_counts())
    print("-" * 40)


Columna: gidenpac
Cantidad de cada valor:
gidenpac
RDST194104612    3
PTGT192301690    2
LVLL192407550    2
GRDZ193702420    2
BRFR193407480    2
FNJS193412230    2
GNMR193709600    2
CSBR193605420    2
LRRD195708160    2
STCR192206470    2
TRGN193106560    2
VLFR192603610    2
VGGR192309590    2
SLLN193101690    2
CSBR193504540    2
DLFR192905170    2
RDCN194811450    2
GRDC193808480    2
LLGR192202500    2
LVFR193112510    2
LNSL192712310    2
TRBD192804680    2
GNLP193109611    2
GZHR193609520    2
CSDM198210580    2
CMCR192706670    2
FLVL193809490    2
PRFR193102450    2
STRD193010450    2
RDGN193709020    2
BNGR193408260    1
SRLZ193806660    1
LPBL193905640    1
MRRS194101410    1
GRFR192905150    1
MRGN192710110    1
GNSH200101010    1
RDST194108590    1
LVFR193601490    1
SPSN194209640    1
FRGR194502660    1
FRPR192709410    1
GRFL193508430    1
CBDL192901430    1
GRGR192903490    1
DZRB193503630    1
LPFR193407460    1
DZFR193702560    1
GNRX193504650    1
CBGR193603280    1

In [179]:

df2 = df.drop(columns=['riesgo_caida'])
df2 = df2.drop(columns=['braden'])
df2 = df2.drop(columns=['Barthel'])
df2 = df2.drop(columns=['movilidad'])
df2.to_csv('DATOS_PREPROCESADOS_2_.csv', index=False)


In [180]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   gidenpac                       924 non-null    object        
 1   itipsexo                       924 non-null    category      
 2   fllegada                       924 non-null    datetime64[ns]
 3   gsitalta                       924 non-null    category      
 4   iotrocen                       924 non-null    category      
 5   gdiagalt                       924 non-null    category      
 6   ds_izq_der                     924 non-null    category      
 7   ds_turno                       924 non-null    category      
 8   ds_edad                        924 non-null    int64         
 9   ds_estancia                    924 non-null    int64         
 10  ds_pre_oper                    924 non-null    int64         
 11  ds_post_oper       

In [181]:
df2.head()

,gidenpac,itipsexo,fllegada,gsitalta,iotrocen,gdiagalt,ds_izq_der,ds_turno,ds_edad,ds_estancia,...,ds_vitamina_d,ds_insuficiencia_respiratoria,ds_insuficiencia_cardiaca,ds_deterioro_cognitivo,ds_insuficiencia_renal,ds_HTA,ds_diabetes,ds_comorbilidades,ds_ulcera_presion,gpersoli
0,GRGT192408040,0,2024-04-11,2,1,S72.102A,0,2,100,27,...,1,1,1,0,0,0,1,4,1,12369062
1,FRMR193008300,0,2023-06-19,2,1,S72.011A,1,1,94,15,...,1,0,0,1,0,1,1,4,1,12368073
2,GRPS193212171,0,2021-02-01,2,1,S72.011A,1,2,92,7,...,0,0,0,0,0,0,0,0,0,12394686
3,SNRT193308080,0,2023-07-02,2,1,S72.142A,0,1,91,11,...,0,0,1,0,1,0,1,3,1,12369062
4,SNCN193604060,0,2023-08-30,2,1,S72.141A,1,0,88,8,...,0,0,0,0,0,0,0,0,0,12368073


In [182]:
import pandas as pd

# Convertir la columna 'fllegada' a datetime, especificando el formato aaaa-mm-dd
df2['fllegada'] = pd.to_datetime(df2['fllegada'], format='%Y-%m-%d')

# Filtrar eliminando las filas con año 2020 o 2021
df2_filtrado = df2[~df2['fllegada'].dt.year.isin([2020, 2021])]

# Actualizar el dataframe original si se desea
df2 = df2_filtrado
df2.to_csv('DATOS_PRPROCESADOS_2_SIN_COVID.csv', index=False)


In [183]:
import pandas as pd

# Convertir la columna 'fllegada' a datetime, especificando el formato aaaa-mm-dd
df2['fllegada'] = pd.to_datetime(df2['fllegada'], format='%Y-%m-%d')

# Filtrar eliminando las filas con año 2020 o 2021
df2_filtrado = df2[df2['fllegada'].dt.year.isin([2022, 2023, 2024])]

# Actualizar el dataframe original si se desea
df2 = df2_filtrado
df2.to_csv('DATOS_PRPROCESADOS_2_2022-2024.csv', index=False)


In [184]:
df2.head()

,gidenpac,itipsexo,fllegada,gsitalta,iotrocen,gdiagalt,ds_izq_der,ds_turno,ds_edad,ds_estancia,...,ds_vitamina_d,ds_insuficiencia_respiratoria,ds_insuficiencia_cardiaca,ds_deterioro_cognitivo,ds_insuficiencia_renal,ds_HTA,ds_diabetes,ds_comorbilidades,ds_ulcera_presion,gpersoli
0,GRGT192408040,0,2024-04-11,2,1,S72.102A,0,2,100,27,...,1,1,1,0,0,0,1,4,1,12369062
1,FRMR193008300,0,2023-06-19,2,1,S72.011A,1,1,94,15,...,1,0,0,1,0,1,1,4,1,12368073
3,SNRT193308080,0,2023-07-02,2,1,S72.142A,0,1,91,11,...,0,0,1,0,1,0,1,3,1,12369062
4,SNCN193604060,0,2023-08-30,2,1,S72.141A,1,0,88,8,...,0,0,0,0,0,0,0,0,0,12368073
5,ZQRS193604120,0,2024-02-16,2,1,S72.141A,1,2,88,10,...,0,0,0,1,0,1,0,3,0,9756465


In [185]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 489 entries, 0 to 922
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   gidenpac                       489 non-null    object        
 1   itipsexo                       489 non-null    category      
 2   fllegada                       489 non-null    datetime64[ns]
 3   gsitalta                       489 non-null    category      
 4   iotrocen                       489 non-null    category      
 5   gdiagalt                       489 non-null    category      
 6   ds_izq_der                     489 non-null    category      
 7   ds_turno                       489 non-null    category      
 8   ds_edad                        489 non-null    int64         
 9   ds_estancia                    489 non-null    int64         
 10  ds_pre_oper                    489 non-null    int64         
 11  ds_post_oper            

In [186]:
for col in df2.columns:
    print(f"Columna: {col}")
    print("Cantidad de cada valor:")
    print(df2[col].value_counts())
    print("-" * 40)


Columna: gidenpac
Cantidad de cada valor:
gidenpac
RDST194104612    3
BRFR193407480    2
DLFR192905170    2
PTGT192301690    2
TRGN193106560    2
CSBR193605420    2
PRFR193102450    2
RDGN193709020    2
FNJS193412230    2
LVFR193112510    2
RDCN194811450    2
GNMR193709600    2
GRGT192408040    1
MRCB193605450    1
BRGN193710480    1
LPCR194104660    1
GRNR193710660    1
GRMR193812630    1
CDGT193504480    1
SPMR193907710    1
MNSN194210670    1
GRBT193907620    1
FRSL193607540    1
TSBL193803610    1
LNGT193702500    1
LRDM193203640    1
RBLV193609590    1
VDMR193412570    1
BTPD193609600    1
GLHR193504500    1
TRPR193403680    1
CRFR193211620    1
VLGN193010580    1
RDRD194309580    1
SPSH194204540    1
MLHD196410700    1
MRRV000000013    1
CLLN193702420    1
JNVD193608640    1
CRGN193309440    1
LVND193010630    1
PLGM193005480    1
PZPR194303470    1
GRGR20020242F    1
DZBR193807630    1
RDCS193403480    1
BRVL193403431    1
CNFR193101470    1
RDRD194109471    1
SNGR193609641    1

In [187]:
# Elimina filas que contengan "-999" o -999 en cualquier columna
df3 = df[~df.isin(["-999", -999]).any(axis=1)]

#Eliminar columnas sin datos
df3 = df3.drop(columns=['iotrocen'])
df3["gdiagalt"] = df3["gdiagalt"].cat.remove_unused_categories()
df3["movilidad"] = df3["movilidad"].cat.remove_unused_categories()
df3["riesgo_caida"] = df3["riesgo_caida"].cat.remove_unused_categories()


df3.to_csv('DATOS_PREPROCESADOS_3.csv', index=False)


In [188]:
import pandas as pd

# Convertir la columna 'fllegada' a datetime, especificando el formato aaaa-mm-dd
df3['fllegada'] = pd.to_datetime(df3['fllegada'], format='%Y-%m-%d')

df3_filtrado = df3[~df3['fllegada'].dt.year.isin([2020, 2021])]

# Actualizar el dataframe original si se desea
df3 = df3_filtrado

df3.to_csv('DATOS_PRPROCESADOS_3_SIN_COVID.csv', index=False)


In [189]:
import pandas as pd

# Convertir la columna 'fllegada' a datetime, especificando el formato aaaa-mm-dd
df3['fllegada'] = pd.to_datetime(df3['fllegada'], format='%Y-%m-%d')

df3_filtrado = df3[df3['fllegada'].dt.year.isin([2022, 2023, 2024])]

# Actualizar el dataframe original si se desea
df3 = df3_filtrado

df3.to_csv('DATOS_PREPROCESADOS_3_2022_2024.csv', index=False)


In [190]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 587 to 587
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   gidenpac                       1 non-null      object        
 1   itipsexo                       1 non-null      category      
 2   fllegada                       1 non-null      datetime64[ns]
 3   gsitalta                       1 non-null      category      
 4   gdiagalt                       1 non-null      category      
 5   ds_izq_der                     1 non-null      category      
 6   ds_turno                       1 non-null      category      
 7   ds_edad                        1 non-null      int64         
 8   ds_estancia                    1 non-null      int64         
 9   ds_pre_oper                    1 non-null      int64         
 10  ds_post_oper                   1 non-null      int64         
 11  ds_vivo_alta            

In [191]:
for col in df3.columns:
    print(f"Columna: {col}")
    print("Cantidad de cada valor:")
    print(df3[col].value_counts())
    print("-" * 40)


Columna: gidenpac
Cantidad de cada valor:
gidenpac
GRGT193902410    1
Name: count, dtype: int64
----------------------------------------
Columna: itipsexo
Cantidad de cada valor:
itipsexo
1    1
0    0
Name: count, dtype: int64
----------------------------------------
Columna: fllegada
Cantidad de cada valor:
fllegada
2022-09-07    1
Name: count, dtype: int64
----------------------------------------
Columna: gsitalta
Cantidad de cada valor:
gsitalta
2    1
1    0
7    0
9    0
Name: count, dtype: int64
----------------------------------------
Columna: gdiagalt
Cantidad de cada valor:
gdiagalt
S72.141A    1
S72.011A    0
S72.012A    0
S72.142A    0
Name: count, dtype: int64
----------------------------------------
Columna: ds_izq_der
Cantidad de cada valor:
ds_izq_der
1    1
0    0
Name: count, dtype: int64
----------------------------------------
Columna: ds_turno
Cantidad de cada valor:
ds_turno
1    1
0    0
2    0
Name: count, dtype: int64
----------------------------------------
Co

Revisando dataset


In [192]:
# Contar el número de valores únicos en cada columna
unique_counts = df.nunique()
print(unique_counts)

gidenpac                         893
itipsexo                           2
fllegada                         697
gsitalta                           4
iotrocen                           2
gdiagalt                           8
ds_izq_der                         2
ds_turno                           3
ds_edad                           50
ds_estancia                       40
ds_pre_oper                       22
ds_post_oper                      37
ds_vivo_alta                       2
ds_dia_semana_llegada              7
ds_mes_llegada                    12
ds_centro_afueras                  2
ds_alergia_medicamentosa           2
ds_alergia_alimenticia             2
ds_otras_alergias                  2
movilidad                          3
Barthel                           22
braden                            14
riesgo_caida                      11
ds_ITU                             2
ds_anemia                          2
ds_vitamina_d                      2
ds_insuficiencia_respiratoria      2
d

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   gidenpac                       924 non-null    object        
 1   itipsexo                       924 non-null    category      
 2   fllegada                       924 non-null    datetime64[ns]
 3   gsitalta                       924 non-null    category      
 4   iotrocen                       924 non-null    category      
 5   gdiagalt                       924 non-null    category      
 6   ds_izq_der                     924 non-null    category      
 7   ds_turno                       924 non-null    category      
 8   ds_edad                        924 non-null    int64         
 9   ds_estancia                    924 non-null    int64         
 10  ds_pre_oper                    924 non-null    int64         
 11  ds_post_oper       

In [194]:
for col in df.columns:
    valores_unicos = df[col].unique()       # Obtiene los valores únicos de la columna
    cantidad_unicos = df[col].nunique()       # Cuenta cuántos valores únicos hay
    print(f"Columna: {col} - Valores únicos: {cantidad_unicos}")
    print(valores_unicos)
    print("-" * 40)


Columna: gidenpac - Valores únicos: 893
['GRGT192408040' 'FRMR193008300' 'GRPS193212171' 'SNRT193308080'
 'SNCN193604060' 'ZQRS193604120' 'PRLV192810280' 'LZDZ193004030'
 'LNGN194608080' 'NSFR194801090' 'FRBL195103090' 'DRCL192405020'
 'NGCL192704250' 'FRCL193109160' 'FRMR193705310' 'FRMR000000081'
 'LRGR194103270' 'LZRZ192504670' 'BRMR192711630' 'SNMG192802420'
 'RMSN191904580' 'PSSN192007680' 'RCLP192404410' 'LVLL192407550'
 'LPVD192612620' 'FRPR192711530' 'PBGR192805690' 'MNFR192806630'
 'CTMR192811100' 'GNML193504080' 'FRRD193511170' 'CLLN193808090'
 'VLRS194509030' 'GRMR192109670' 'PBSN193607100' 'CNCM192602040'
 'LVTS192811150' 'LVPR192906080' 'PRDP193009080' 'GRGN193503260'
 'LLBD195110130' 'DZLV195612070' 'RBCS192905450' 'FRHL193005530'
 'PRLZ193201540' 'FLGR193205680' 'GRGR193302590' 'PRPM193311530'
 'GRLV193610710' 'FRBR192703680' 'VZVZ192705700' 'MLFR192811680'
 'FRGR193109290' 'FNPR193308020' 'LPPR193504220' 'SPGR193507290'
 'GRFR193612110' 'RDBR000000058' 'DLFR192905170' '

In [195]:
for col in df.columns:
    print(f"Columna: {col}")
    print("Cantidad de cada valor:")
    print(df[col].value_counts())
    print("-" * 40)


Columna: gidenpac
Cantidad de cada valor:
gidenpac
RDST194104612    3
PTGT192301690    2
LVLL192407550    2
GRDZ193702420    2
BRFR193407480    2
FNJS193412230    2
GNMR193709600    2
CSBR193605420    2
LRRD195708160    2
STCR192206470    2
TRGN193106560    2
VLFR192603610    2
VGGR192309590    2
SLLN193101690    2
CSBR193504540    2
DLFR192905170    2
RDCN194811450    2
GRDC193808480    2
LLGR192202500    2
LVFR193112510    2
LNSL192712310    2
TRBD192804680    2
GNLP193109611    2
GZHR193609520    2
CSDM198210580    2
CMCR192706670    2
FLVL193809490    2
PRFR193102450    2
STRD193010450    2
RDGN193709020    2
BNGR193408260    1
SRLZ193806660    1
LPBL193905640    1
MRRS194101410    1
GRFR192905150    1
MRGN192710110    1
GNSH200101010    1
RDST194108590    1
LVFR193601490    1
SPSN194209640    1
FRGR194502660    1
FRPR192709410    1
GRFL193508430    1
CBDL192901430    1
GRGR192903490    1
DZRB193503630    1
LPFR193407460    1
DZFR193702560    1
GNRX193504650    1
CBGR193603280    1

In [196]:
# Prueba
import pandas as pd

# Ajusta las opciones de visualización para mostrar todas las filas
pd.set_option('display.max_rows', None)

# O, si prefieres, convierte el resultado a string completo
conteo = df_valoracion_enfermeria["dconclin"].value_counts()
print(conteo.to_string())


KeyError: 'dconclin'